In [1]:
from iroha import IrohaCrypto
from iroha.ed25519 import H
import integration_helpers
from iroha.primitive_pb2 import can_set_my_account_detail
import json
from iroha_helper import *
from new_helper import *
from super_helper import *
from ipfs_functions import *
from loguru import logger


In [2]:
# Index for objects in both user account and project account JSON-LDs.
json_ld_index = 0


# Local path for file upload
directory_path = "upload"

In [3]:
#Manually resets the index on execution
# recreate_index() 

1 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [4]:
hash = create_contract()
integration_helpers.get_engine_receipts_result(hash)

	Entering "create_contract"
('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "create_contract"
	Entering "get_engine_receipts_result"

	Leaving "get_engine_receipts_result"


2 - Data extraction from JSON-LD.

Extracts account ids from `datasets/accounts.json` and `datasets/projects.json`.

Must update `json_ld_index` with a entry number related to an existing object in `datasets/accounts.json` and `datasets/projects.json`

4 - Sets details for both User and Project accounts providing a logical link between them for later references.

In [5]:
# Example execution of the previous snippet
address = integration_helpers.get_engine_receipts_address(hash)

# Read accounts from JSON-LD
user_accounts = read_user_accounts_from_jsonld('datasets/accounts.json')
project_accounts = read_project_accounts_from_jsonld('datasets/projects.json')

# Assuming json_ld_index is defined
user_account = user_accounts[json_ld_index]
project_account = project_accounts[json_ld_index]

# Set project_id as a detail for the user account
hash_user_to_project = set_account_detail(
    address, 
    user_account['account_id'], 
    "linked_project", 
    project_account['account_id']
)

# Set user_account_id as a detail for the project account
hash_project_to_user = set_account_detail(
    address, 
    project_account['account_id'], 
    "linked_user", 
    user_account['account_id']
)

# Update the JSON-LD files with the linked details
update_user_account_link(user_account['account_id'], project_account['account_id'])
update_project_account_link(project_account['account_id'], user_account['account_id'])

# Confirming the operation
logger.info(f"User account {user_account['account_id']} linked to project {project_account['account_id']}")
logger.info(f"Project account {project_account['account_id']} linked to user {user_account['account_id']}")

	Entering "get_engine_receipts_address"
	Leaving "get_engine_receipts_address"
	Entering "set_account_detail"
None
('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "set_account_detail"
	Entering "set_account_detail"
None
('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)


2025-01-09 18:35:41.798 | INFO     | __main__:<module>:33 - User account competent_chandrasekhar@test linked to project 14427@test
2025-01-09 18:35:41.800 | INFO     | __main__:<module>:34 - Project account 14427@test linked to user competent_chandrasekhar@test


('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "set_account_detail"
Updated user account competent_chandrasekhar@test with linked project 14427@test
Updated project account 14427@test with linked user competent_chandrasekhar@test


3 - Queries Iroha 1 for User account and checks its values

In [6]:
#Query - GetAccountDetail
query = iroha.query('GetAccountDetail',account_id=user_account['account_id'])
# logger.info(query)
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
# logger.info(response)

user_data = response.account_detail_response
user_details = user_data.detail


logger.info(f'User Account id = {user_account}, {user_details}')



2025-01-09 18:35:41.837 | INFO     | __main__:<module>:12 - User Account id = {'account_id': 'competent_chandrasekhar@test'}, { "admin@test" : { "linked_project" : "14427@test", "user_json_ld_cid" : "QmV1Xsfi7AZBbim46cz8VQTTw3sYUQWkQWaz7kB6Artqg1" } }


6 - Queries the user account, locates the project id, queries the project account, gets the metadata and files from IPFS.

In [7]:
# Process the account details response
user_details_dict = json.loads(user_details)  # Convert the string to a JSON object
logger.info(user_details_dict)

# Now you can access the specific key like this
project_id = user_details_dict["admin@test"]["linked_project"]
logger.info(project_id)

2025-01-09 18:35:41.893 | INFO     | __main__:<module>:3 - {'admin@test': {'linked_project': '14427@test', 'user_json_ld_cid': 'QmV1Xsfi7AZBbim46cz8VQTTw3sYUQWkQWaz7kB6Artqg1'}}
2025-01-09 18:35:41.896 | INFO     | __main__:<module>:7 - 14427@test


In [8]:
schema = get_schema() #super_helper.py

logger.info(schema)

processed_data = process_files(directory_path, project_id, schema) #new_helper.py
 
    

2025-01-09 18:35:41.916 | INFO     | __main__:<module>:3 - <Schema: ['abstract', 'created', 'creator', 'date', 'description', 'file_cid', 'format', 'full_text', 'language', 'metadata_cid', 'modified', 'project_id', 'publisher', 'subject', 'title']>


	Entering "create_contract"
('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "create_contract"
	Entering "get_engine_receipts_address"
	Leaving "get_engine_receipts_address"
	Entering "set_account_detail"
None
('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)


2025-01-09 18:35:47.902 | INFO     | super_helper:setup_index:328 - Opened existing index.
2025-01-09 18:35:48.057 | INFO     | super_helper:index_metadata:186 - Metadata indexed successfully: Qmda3UGvN1Ywmc2FkxuKSEHK8iy5vW5YrRFE9sqKKxdUVw


('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "set_account_detail"


In [9]:
account_detail = get_account_detail(project_id)
logger.info(f"{project_id}, {account_detail}")


2025-01-09 18:35:48.067 | INFO     | __main__:<module>:2 - 14427@test, { "admin@test" : { "file_1" : "QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, Qmda3UGvN1Ywmc2FkxuKSEHK8iy5vW5YrRFE9sqKKxdUVw", "linked_user" : "competent_chandrasekhar@test", "project_metadata_cid" : "QmWCXQ2erWT5tn5HGhy3othDfv2QQnatYh9jrURY5GMdS9" } }


In [10]:
# Open an existing index (adjust the path to your index directory)
index_path = "indexdir"
index = open_dir(index_path)

# Directory for file downloads
download_path = "download"

# Perform a keyword search
keyword = "must"
search_results, project_ids_with_cids = search_index(index, keyword)

# Check if there are no search results
if not search_results:
    logger.warning(f"No search results found for keyword: '{keyword}'. Exiting the script.")
else:
    # Process each dictionary in search results
    for result_dict in search_results:
        project_id = result_dict.get('project_id')
        file_cid = result_dict.get('file_cid')
        metadata_cid = result_dict.get('metadata_cid')

        if not project_id or not file_cid or not metadata_cid:
            logger.error(f"Missing required data in result: {result_dict}")
            continue

        # Log the retrieved project details
        logger.info(f"Processing Project ID: {project_id}")
        logger.debug(f"File CID: {file_cid}")
        logger.debug(f"Metadata CID: {metadata_cid}")
        # file_metadata_json = download_json_from_ipfs(metadata_cid)
        # logger.info("file_metadata_json:", file_metadata_json)
        

        # Fetch project details from the blockchain
        project_details = get_account_detail(project_id)
        if not project_details:
            logger.error(f"No project details found for Project ID: {project_id}.")
            continue

        logger.debug(f"Fetched project details for {project_id}: {project_details}")

        # Parse blockchain data
        try:
            blockchain_data = json.loads(project_details)
        except json.JSONDecodeError as e:
            logger.error(f"Error decoding project details JSON for {project_id}: {e}")
            continue

        # Validate file CID and fetch project details
        validation_result = fetch_project_details(file_cid, blockchain_data)
        if validation_result["is_valid"]:
            project_metadata_cid = validation_result.get("project_metadata_cid")
            linked_user = validation_result.get("linked_user")
            file_metadata_cid = validation_result.get("metadata_cid")
            
            # download_file(file_metadata_json, download_path, project_id, file_cid)

            logger.info(f"Valid File CID for {project_id}.")
            logger.debug(f"Project Metadata CID: {project_metadata_cid}")
            logger.debug(f"Linked User: {linked_user}")

            # Fetch and process metadata and user details
            if project_metadata_cid:
                logger.debug(f"Processing project metadata CID: {project_metadata_cid}")
                project_metadata = download_json_from_ipfs(project_metadata_cid)
                logger.debug(f"Downloaded project metadata: {project_metadata}")

            if linked_user:
                logger.debug(f"Processing linked user: {linked_user}")
                user_details = get_account_detail(linked_user)
                try:
                    user_details = json.loads(user_details)
                except json.JSONDecodeError as e:
                    logger.error(f"Error decoding user details JSON for {linked_user}: {e}")
                    continue

                user_json_ld_cid = user_details.get("admin@test", {}).get("user_json_ld_cid", None)
                if user_json_ld_cid:
                    logger.info(f"User JSON-LD CID: {user_json_ld_cid}")
                    user_metadata = download_json_from_ipfs(user_json_ld_cid)
                    logger.debug(f"Downloaded user metadata: {user_metadata}")
                else:
                    logger.warning(f"User JSON-LD CID not found for linked user {linked_user}.")
            
            if metadata_cid:
                logger.info(f"Processing metadata CID: {metadata_cid}")
                file_metadata = download_json_from_ipfs(metadata_cid)
                file_metadata_json = download_file(file_metadata, download_path, project_id, file_cid)
                logger.info(f"Downloaded file metadata: {metadata_cid}")
                logger.info(f"file metadata: {file_metadata}")
                logger.info(file_metadata_json)
            
        
        else:
            logger.warning(f"Invalid File CID for Project ID: {project_id}. Skipping metadata processing.")


2025-01-09 18:35:48.153 | INFO     | super_helper:search_index:204 - Starting keyword search...
2025-01-09 18:35:48.155 | INFO     | super_helper:search_index:205 - Keyword: 'must'
2025-01-09 18:35:48.189 | INFO     | super_helper:search_index:216 - Search successful: Found 15 result(s).
2025-01-09 18:35:48.193 | INFO     | super_helper:search_index:218 - 1. Project Id: 06535@test, File CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, Metadata CID: QmTf2AqDtce9DUSDxNnkGFkwAX9uVTYB2iQfJtxubZZEVx, Title: covid19-mlsf: a multi-task learning-based stock market forecasting framework during the covid-19 pandemic
2025-01-09 18:35:48.196 | INFO     | super_helper:search_index:218 - 2. Project Id: 06535@test, File CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, Metadata CID: QmUDmBpFAf5Ns5VsNigNNWJmwLSS6b4Y2DRbvEdSaDFCuv, Title: covid19-mlsf: a multi-task learning-based stock market forecasting framework during the covid-19 pandemic
2025-01-09 18:35:48.200 | INFO     | super_helper:sea